Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [20]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os

First reload the data we generated in `1_notmnist.ipynb`.

In [21]:
#pickle_file = "~/deeplearning/assignment/assignment1/notMNIST.pickle"
#open(os.path.expanduser("~/deeplearning/assignment/assignment1/notMNIST.pickle"), "rb")
with open(os.path.expanduser("~/deeplearning/assignment/assignment1/notMNIST.pickle"), "rb") as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [22]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [23]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [24]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable( tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases

    #Loss
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(_sentinel=None,labels=tf_train_labels,logits=logits) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [25]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 43.969135
Minibatch accuracy: 11.7%
Validation accuracy: 14.0%
Minibatch loss at step 500: 0.688133
Minibatch accuracy: 85.2%
Validation accuracy: 82.1%
Minibatch loss at step 1000: 0.615548
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 0.838513
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.774884
Minibatch accuracy: 82.8%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 0.696651
Minibatch accuracy: 82.8%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 0.933139
Minibatch accuracy: 79.7%
Validation accuracy: 81.9%
Test accuracy: 87.5%


In [26]:
k=len(train_dataset)
print ('',train_dataset.shape )

 (200000, 784)


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [27]:
batch_size = 100
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  



    # Variables.

    weights = tf.Variable( tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases

    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(_sentinel=None,labels=tf_train_labels,logits=logits) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [28]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.632767
Minibatch accuracy: 9.0%
Validation accuracy: 13.1%
Minibatch loss at step 500: 0.719783
Minibatch accuracy: 83.0%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.836284
Minibatch accuracy: 79.0%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.808190
Minibatch accuracy: 81.0%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 0.921262
Minibatch accuracy: 76.0%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.710456
Minibatch accuracy: 84.0%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.734262
Minibatch accuracy: 83.0%
Validation accuracy: 81.3%
Test accuracy: 87.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [29]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable( tf.truncated_normal([image_size * image_size, num_labels]) )
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits=tf.matmul(tf.nn.dropout(tf_train_dataset,0.25),weights)+biases
    #logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(_sentinel=None,labels=tf_train_labels,logits=logits) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [30]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 63.453735
Minibatch accuracy: 11.7%
Validation accuracy: 14.5%
Minibatch loss at step 500: 0.828324
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 0.983699
Minibatch accuracy: 68.8%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 1.115274
Minibatch accuracy: 68.0%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 1.139767
Minibatch accuracy: 74.2%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 1.104116
Minibatch accuracy: 70.3%
Validation accuracy: 80.7%
Minibatch loss at step 3000: 1.288907
Minibatch accuracy: 73.4%
Validation accuracy: 80.3%
Test accuracy: 86.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [33]:
#Multilayer model!
#Use 2 layers

batch_size = 128
hidden_size1 = 1024
hidden_size2 = 100
beta_regul = 0.9e-3

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
    # Variables.
    #weights are then divided by stddev I don't know why
    weights = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_size1], stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases = tf.Variable(tf.zeros([hidden_size1]))
    
    
    weights2 = tf.Variable( tf.truncated_normal([hidden_size1, hidden_size2], stddev=np.sqrt(2.0 / (hidden_size1))))
    biases2 = tf.Variable(tf.zeros([hidden_size2]))
    
    weights3 = tf.Variable( tf.truncated_normal([hidden_size2, num_labels], stddev=np.sqrt(2.0 / (hidden_size2))))
    biases3 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    
    layer_1_train = tf.nn.relu(tf.matmul(tf_train_dataset,weights)+biases)
    layer_2_train = tf.nn.relu(tf.matmul(layer_1_train,weights2)+biases2)
    logits = tf.matmul(layer_2_train,weights3)+biases3

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) + \
      beta_regul * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  
    # Optimizer.
    
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    

    train_prediction = tf.nn.softmax(logits)
    
    layer_valid1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
    layer_valid2 = tf.nn.relu(tf.matmul(layer_valid1, weights2) + biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(layer_valid2, weights3) + biases3)
    
    layer_test1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    layer_test2 = tf.nn.relu(tf.matmul(layer_test1, weights2) + biases2)
    test_prediction = tf.nn.softmax(tf.matmul(layer_test2, weights3) + biases3)

In [34]:
num_steps = 9001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        #Minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.210423
Minibatch accuracy: 9.4%
Validation accuracy: 33.4%
Minibatch loss at step 500: 0.842028
Minibatch accuracy: 91.4%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.672860
Minibatch accuracy: 91.4%
Validation accuracy: 87.2%
Minibatch loss at step 1500: 0.688267
Minibatch accuracy: 89.8%
Validation accuracy: 88.3%
Minibatch loss at step 2000: 0.593452
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 2500: 0.522155
Minibatch accuracy: 92.2%
Validation accuracy: 88.8%
Minibatch loss at step 3000: 0.626068
Minibatch accuracy: 89.1%
Validation accuracy: 89.3%
Minibatch loss at step 3500: 0.421416
Minibatch accuracy: 92.2%
Validation accuracy: 89.6%
Minibatch loss at step 4000: 0.588636
Minibatch accuracy: 87.5%
Validation accuracy: 89.8%
Minibatch loss at step 4500: 0.471194
Minibatch accuracy: 91.4%
Validation accuracy: 90.3%
Minibatch loss at step 5000: 0.503710
Minibatch accuracy: 89.1%
Validation accuracy